Part-II of week4
1. Load the movie ratings data (as in the HW3-recommender-system) and use matrix factorization technique(s) and predict the missing ratings from the test data. Measure the RMSE. You should use sklearn library.
Make sure that your notebook includes the following:
use's sklearn's non-negative matrix factorization

In [57]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.decomposition import NMF
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [58]:
users = pd.read_csv('./users.dat', sep='::', engine='python', header=None, 
                    names=['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'])
movies = pd.read_csv('./movies.dat', sep='::', engine='python', header=None, names=['MovieID', 'Title', 'Genres'])
ratings = pd.read_csv('./ratings.dat', sep='::', engine='python',
                      names=['UserID', 'MovieID', 'Rating', 'Timestamp'])

In [59]:
users.head()

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [60]:
movies.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [61]:
ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [62]:
le_movie = LabelEncoder()
movies['MovieID'] = le_movie.fit_transform(movies['MovieID'])

le_user = LabelEncoder()
ratings['UserID'] = le_user.fit_transform(ratings['UserID'])

In [63]:
ratings_matrix = ratings.pivot_table(index='UserID', columns='MovieID', values='Rating').fillna(0)
ratings_matrix

MovieID,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
UserID,,,,,,,,,,,,,,,,,,,,,
0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6036,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [64]:
X_train, X_test = train_test_split(ratings_matrix, test_size=0.2, random_state=42)

In [65]:
nmf = NMF(n_components=5,random_state=42)
nmf.fit(X_train)
predicted_ratings = nmf.transform(X_test)
predicted_ratings = np.dot(predicted_ratings, nmf.components_)
predicted_ratings[predicted_ratings < 0] = 0

In [69]:
test_rmse = np.sqrt(mean_squared_error(X_test[X_test != 0], predicted_ratings[X_test != 0]))
print(f'Test RMSE: {test_rmse:.2f}')

Test RMSE: 2.75


2. Discuss the results and why they did not work well compared to simple baseline or similarity-based methods we’ve done in Module 3. Can you suggest a way(s) to fix it?

RMSE of 2.75 is significantly higher than all the RMSE values listed in the table from week 3. This means that this model's predictions are less accurate than those of the other methods listed in week 3. The reason could be that the algorithm used to calculate similarity between users or items is not optimal for this particular dataset. I did fill in the null/NaN values with 0. I could have imputed the values with the mean or found a better solution to handle them. This could have affected the predictions greatly. Additionally, the hyperparameters of the model may not have been tuned properly. We could try increasing the n_companonents. By increasing the number of components, maybe we could have captured more complex patterns in the data and potentially improved the accuracy of the model.